In [5]:
from dash_application.data import data
from dash_application.input_data import input_data
import plotly
import pandas as pd
from dash_application.plots import report_plots
sme_main = input_data()

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
import calendar
import datetime


import pandas as pd
import datetime
from datetime import datetime
import calendar


import psycopg2




def input_data():
    month = str(datetime.now().strftime('%B'))
    # Establish a connection to the database by creating a cursor object

    # Obtain the configuration parameters
    params = config()
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # Create a new cursor
    cur = conn.cursor()

    # A function that takes in a PostgreSQL query and outputs a pandas database 
    def create_pandas_table(sql_query, database = conn):
        table = pd.read_sql_query(sql_query, database)
        return table

    # Utilize the create_pandas_table function to create a Pandas data frame
    # Store the data as a variable
    sme_main = create_pandas_table("SELECT * FROM sme_main")


    #sme_main = pd.read_excel('sme_sample2.xlsx')
    # Close the cursor and connection to so the server can allocate
    # bandwidth to other requests
    cur.close()
    conn.close()


    sme_main['order_date']=pd.to_datetime(sme_main['order_date'],format='%Y-%m-%d')



    sme_main['day'] = pd.DatetimeIndex(sme_main['order_date']).day
    sme_main['year'] = pd.DatetimeIndex(sme_main['order_date']).year
    sme_main['month']= pd.DatetimeIndex(sme_main['order_date']).month
    sme_main['week'] = (sme_main['order_date'].dt.strftime('%W').astype(int) )
    sme_main['month_name'] = sme_main['month'].apply(lambda x: calendar.month_name[x])
    sme_main['driver_fee'] = sme_main['order_delivery_fees']*0.7
    sme_main['halan_return'] = sme_main['order_delivery_fees']*0.3
    sme_main['sme_return'] = sme_main['order_value'] - sme_main['order_delivery_fees']


    return sme_main
from configparser import ConfigParser


def config(filename=r'C:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Semester 10\FYP\Halan_Dashboard\dash_application\database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db




latest = input_data()['order_id'].max()
print(latest)
print(datetime.today())
w = datetime.today().isocalendar()[1]
d = datetime.today().isocalendar()[2]//7 +1
print(w,d)





272044
2022-03-11 17:17:07.659681
10 1


c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
print(f'{w}{d}{int(latest[-3:])+1}')

10145


In [6]:
halan_regression3 = pd.read_csv('dash_application/csv/weekly_data2.csv')

halan_regression3.head(100)

,Month,w1,w2,w3,w4
0,6,509.000000,388.000000,405.00000,343.000000
1,7,553.000000,342.000000,353.00000,293.000000
2,8,373.000000,375.000000,514.00000,649.000000
3,9,630.000000,450.000000,500.00000,700.000000
4,10,520.000000,537.000000,410.00000,159.000000
5,11,461.613733,415.262743,417.59703,412.646005


In [56]:
halan_regression3['average']=halan_regression3.iloc[:,1:5].mean(axis=1)
df = halan_regression3.copy()
df['w1']=df['w1']/df['average']
df['w2']=df['w2']/df['average']
df['w3']=df['w3']/df['average']
df['w4']=df['w4']/df['average']

columns = ['w1','w2','w3','w4']
deseasonalized = pd.DataFrame()

deseasonalized['Month']=[halan_regression3['Month'][j] for j in range(len(df)) for i in columns]
deseasonalized['Week']=[int(i[-1]) for j in range(len(df)) for i in columns]

#deseasonalized['original']=[halan_regression3[i][j] for j in range(len(df)) for i in columns]
deseasonalized['Delivered'] =[halan_regression3[i][j]/df[i].mean(axis=0) for j in range(len(df)) for i in columns]       
deseasonalized

,Month,Week,Delivered
0,6,1,441.331985
1,6,2,408.446103
2,6,3,416.006596
3,6,4,371.538133
4,7,1,479.482491
5,7,2,360.022081
6,7,3,362.593403
7,7,4,317.378055
8,8,1,323.412241
9,8,2,394.761054


halan_regression3['average']=halan_regression3.iloc[:,1:5].mean(axis=1)
df = halan_regression3.copy()
df['w1']=df['w1']/df['average']
df['w2']=df['w2']/df['average']
df['w3']=df['w3']/df['average']
df['w4']=df['w4']/df['average']

columns = ['w1','w2','w3','w4']
deseasonalized = pd.DataFrame()
deseasonalized['order_date']= [i+1 for i in range(len(columns)*len(df))]
deseasonalized['Delivered'] =[halan_regression3[i][j]/df[i].mean(axis=0) for j in range(len(df)) for i in columns]
deseasonalized['original']=[halan_regression3[i][j] for j in range(len(df)) for i in columns]
        
deseasonalized

In [79]:
# evaluate an ridge regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
# load the dataset

dataframe = deseasonalized
data = dataframe.values

X, y = data[:, :-1], data[:, -1]
print(y)
print(X)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define model
model = RidgeCV(alphas=arange(0, 1, 0.01), cv=cv, scoring='neg_mean_absolute_error')
# fit model
model.fit(X, y)

# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))
print('alpha: %f' % model.alpha_)

[441.3319852  408.44610338 416.00659581 371.53813271 479.4824908
 360.02208082 362.59340326 317.37805506 323.41224063 394.76105353
 527.9688648  702.99780796 546.24587559 473.71326423 513.58838989
 758.24108716 450.8696116  565.29782865 421.14247971 172.22904694
 400.24539311 437.14548725 428.94597231 446.9787936 ]
[[ 6.  1.]
 [ 6.  2.]
 [ 6.  3.]
 [ 6.  4.]
 [ 7.  1.]
 [ 7.  2.]
 [ 7.  3.]
 [ 7.  4.]
 [ 8.  1.]
 [ 8.  2.]
 [ 8.  3.]
 [ 8.  4.]
 [ 9.  1.]
 [ 9.  2.]
 [ 9.  3.]
 [ 9.  4.]
 [10.  1.]
 [10.  2.]
 [10.  3.]
 [10.  4.]
 [11.  1.]
 [11.  2.]
 [11.  3.]
 [11.  4.]]
Mean MAE: 98.261 (59.525)


In [75]:
row = [12, 1]
# make a prediction
yhat = model.predict([row])
# summarize prediction
print('Predicted: %.3f' % yhat)

Predicted: 461.141


In [76]:
from numpy import arange
from sklearn.model_selection import GridSearchCV

# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -98.261
Config: {'alpha': 0.99}


In [86]:
from scipy.stats import pearsonr
corr, _ = pearsonr(X, y)
corr

ValueError: shapes (24,2) and (24,) not aligned: 2 (dim 1) != 24 (dim 0)

In [ ]:
# evaluate an ridge regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
# load the dataset

In [8]:



dataframe = deseasonalized
data = dataframe.values

X, y = data[:, :-1], data[:, -1]
print(y)
print(X)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define model
model = RidgeCV(alphas=arange(0, 1, 0.01), cv=cv, scoring='neg_mean_absolute_error')
# fit model
model.fit(X, y)

# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))
print('alpha: %f' % model.alpha_)

,order_date,Delivered,original
0,1,441.331985,509.000000
1,2,408.446103,388.000000
2,3,416.006596,405.000000
3,4,371.538133,343.000000
4,5,479.482491,553.000000
5,6,360.022081,342.000000
6,7,362.593403,353.000000
7,8,317.378055,293.000000
8,9,323.412241,373.000000
9,10,394.761054,375.000000
